In [4]:
import joblib
import numpy as np

# Carregar o modelo e os scalers
model_filename = 'best_mlp_model.pkl'
scalerx_filename = 'scalerx.pkl'
scalery_filename = 'scalery.pkl'

loaded_model = joblib.load(model_filename)
scalerx = joblib.load(scalerx_filename)
scalery = joblib.load(scalery_filename)

## valores para Reynolds e Stdp
Re = np.linspace(1000, 8000, 100)  
Stdp = np.linspace(0.2992, 0.4331, 100)  

# Empacotar os valores de Re e Stdp para passar ao modelo
X = np.array(list(zip(Re, Stdp)))

# Escalar os dados
X_scaled = scalerx.transform(X)

# Fazer a previsão com o modelo
f = loaded_model.predict(X_scaled).reshape(-1, 1)

# Desescalar a previsão
f_inverse = scalery.inverse_transform(f)

In [10]:
#Experimental DATA import

#import data
import pandas as pd
import os

file_name = 'fxReynoldsV3.xlsx'
file_path = os.path.join(os.getcwd(), file_name)
from sklearn.model_selection import train_test_split


ANNDF = pd.read_excel(file_path, sheet_name='expdata')

Xexp = ANNDF.iloc[:,[0,2]].values
Yexp = ANNDF.iloc[:, 1].values.reshape(-1,1)

Xexptrain, Xexptest, yexptrain,  yexptest = train_test_split(Xexp, Yexp, train_size = 0.85)

#----------- 
#Iyer et al points
def Iyerf(Re):
    fIyer = 15.5 * np.power(Re,-0.58) 
    return fIyer


In [22]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo

# Definir variáveis para controle do estilo
font_family = 'Arial'
font_size = 20  # Fonte geral para títulos dos eixos
legend_font_size = 15  # Fonte da legenda
tick_font_size = 13  # Fonte para os valores dos eixos (ticks)
colorbar_tick_font_size = 15  # Fonte para os valores da barra de cores (colorbar)
show_grid = True  # Controlar se a grade estará ativada ou desativada (True ou False)

# Criar uma grade de Re e Stdp (Assumindo que Re e Stdp já estão definidos)
Re_grid, Stdp_grid = np.meshgrid(Re, Stdp)

# Escalar os pontos da grade
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]
grid_points_scaled = scalerx.transform(grid_points)

# Fazer previsões para cada ponto da grade usando o modelo carregado
f_grid_pred = loaded_model.predict(grid_points_scaled).reshape(-1, 1)

# Desescalar as previsões
f_pred_grid = scalery.inverse_transform(f_grid_pred)

# Ajustar a forma de f_pred_grid para que corresponda à forma de Re_grid e Stdp_grid
f_pred_grid = f_pred_grid.reshape(Re_grid.shape)

# Scatter plot dos valores originais
scatter_data = go.Scatter3d(
    x=Xexptrain[:, 0],  # Valores de Reynolds
    y=Xexptrain[:, 1],  # Valores de Stdp
    z=yexptrain.flatten(),  # Valores de 'f'
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Experimental Data'
)

# Criar a superfície com os resultados da ANN
surface_ann = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_pred_grid,
    colorscale='Rainbow',  # Mudar para tons de arco-íris
    opacity=0.7,  # Aplicar transparência (valor de 0 a 1)
    name='Superfície ANN',
    colorbar=dict(
        title='f',
        len=0.5,
        x=0.9,
        tickfont=dict(
            family=font_family,
            size=colorbar_tick_font_size  # Tamanho da fonte dos valores da barra de cores
        )
    ),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_pred_grid),
            "end": np.max(f_pred_grid),
            "size": (np.max(f_pred_grid) - np.min(f_pred_grid)) / 20,  # Ajustar o espaçamento dos contornos
            "color": "black",
        }
    }
)

# Definir o layout do gráfico com título centralizado e controle de fonte
layout = go.Layout(
    title=dict(
        text='ANN f(Re, Stdp) Surface',
        x=0.5,  # Centraliza o título
        font=dict(
            family=font_family,
            size=font_size + 2,  # Tamanho de fonte um pouco maior para o título
        )
    ),
    scene=dict(
        xaxis=dict(
            title='Reynolds Number (Re)',
            titlefont=dict(family=font_family, size=font_size),
            backgroundcolor='white',
            gridcolor='lightgrey',
            showgrid=show_grid,  # Controlar se a grade será mostrada
            tickfont=dict(family=font_family, size=tick_font_size)  # Fonte para os valores (ticks) do eixo X
        ),
        yaxis=dict(
            title='Stdp',
            titlefont=dict(family=font_family, size=font_size),
            backgroundcolor='white',
            gridcolor='lightgrey',
            showgrid=show_grid,  # Controlar se a grade será mostrada
            tickfont=dict(family=font_family, size=tick_font_size)  # Fonte para os valores (ticks) do eixo Y
        ),
        zaxis=dict(
            title='f',
            titlefont=dict(family=font_family, size=font_size),
            backgroundcolor='white',
            gridcolor='lightgrey',
            showgrid=show_grid,  # Controlar se a grade será mostrada
            tickfont=dict(family=font_family, size=tick_font_size)  # Fonte para os valores (ticks) do eixo Z
        ),
        bgcolor='white'
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        font=dict(
            family=font_family,
            size=legend_font_size  # Fonte da legenda
        )
    )
)

# Criar a figura com ambas as superfícies (ANN e fórmula)
fig = go.Figure(data=[surface_ann, scatter_data], layout=layout)

# Mostrar o gráfico
pyo.plot(fig, auto_open=True)


'temp-plot.html'